In [1]:
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1 import make_axes_locatable
import networkx as nx
import pandas as pd
import scipy as sp
from scipy.stats import lognorm
from scipy.optimize import root_scalar
from scipy.special import gamma
import numpy as np
from os.path import join

# agent based model classes & functionality
import sys
sys.path.insert(0,'../school')
sys.path.insert(0,'../nursing_home')
from model_nursing_home import SEIRX_nursing_home
import analysis_functions as af

# for progress bars
from ipywidgets import IntProgress
from IPython.display import display
import time

In [2]:
def compose_agents(prevention_measures):
    agent_types = {
            'resident':{
                'screening_interval':measures['resident_screen_interval'],
                'index_probability':measures['resident_index_probability']},

            'employee':{
                'screening_interval': measures['employee_screen_interval'],
                'index_probability': measures['employee_index_probability']},
    }
    
    return agent_types

In [3]:
## statistics parameters
# number of maximum steps per run. This is a very conservatively chosen value
# that ensures that an outbreak will always terminate within the allotted time.
# Most runs are terminated way earlier anyways, as soon as the outbreak is over.
N_steps = 100
# number of runs per ensemble
N_runs = 10000
# number of points in the parameter grid that will be randomly sampled
N_samples = 'all'

In [4]:
model_params = {
    # mean and variance of a Weibull distribution characterizing the
    # time between transmission and becoming infectious (in days)
    'exposure_duration':[5.1, 1.9], # literature values
    # mean and variance of a Weibull distribution characterizing the
    # time between transmission and showing symptoms in clinical courses
    # of the infection (in days)
    'time_until_symptoms':[6.4, 0.8], # literature values
    # mean and variance of a Weibull distribution characterizing the
    # time between transmission and ceasing to be infectious (in days)
    'infection_duration':[10.91, 3.95], # literature values
    # modification of the transmission risk in subclinical courses
    'subclinical_modifier':0.6, 
    # base transmission risk of a contact of type "close"
    'base_risk':0.0737411844049918, # artificially high, so you can see stuff happening
    # efficiency of masks (surgical), reducing the transmission risk
    # (exhale) if the source wears a mask and/or the reception risk 
    # (inhale), if the target (also) wears a mask
    'mask_filter_efficiency':{'exhale':0.5, 'inhale':0.7}, # literature values
    # modifiers of the base_risk for transmissions of contact type close
    # if the contact type is "intermediate", "far" or "very var"
    'infection_risk_contact_type_weights':\
        {'very_far':0, 'far':0.75, 'intermediate':0.85,'close':1}, 
    # agent group from which the index case is drawn
    'index_case':'employee',
    # verbosity level (can be 0, 1, 2) that prints increasingly detailed 
    # information about the simulation
    'verbosity':0
}

measures = {
    # enables testing and tracing actions (run with testing=False) to simulate
    # unhindered spread of the virus through the nursing home
    'testing':'diagnostic',
    # test technology and turnover time used for preventive screening
    'preventive_screening_test_type':None,
    # test technology and turnover time used for diagnostic testing
    'diagnostic_test_type':'two_day_PCR',
    # definition of contact types that will be quarantined in case one
    # of the agents in contact had a positive test result
    'K1_contact_types':['close', 'intermediate'],
    # duration (in days) that agents will stay quarantined
    'quarantine_duration':10,
    # interval of a potential follow-up background screen (in days)
    # after a background screen that was initiated by a positive test
    'follow_up_testing_interval':None,
    # whether or not a negative test result "frees" agents from quarantine
    'liberating_testing':False,
    # modification of the transmission risk by ventilation 
    # (1 = no modification, 0.5 = risk is reduced by 50%)
    'ventilation_modification':1, # no ventilation
    'resident_screen_interval':None,
    'employee_screen_interval':None,
    'resident_index_probability':0,
    'employee_index_probability':0
}

In [6]:

# progress bar
f = IntProgress(min=0, max= N_runs) 
display(f)
c = 0


results = pd.DataFrame()

# get the values of the calibration parameters
intermediate_contact_weight, far_contact_weight =[0.4 , 0.2]

# since we only use contacts of type "close", "intermediate" and "far" in 
# this setup, we set the contact type "very far" to 0. The contact type
# "close" corresponds to household transmissions and is set to 1 (= base 
# transmission risk). We therefore only calibrate the weight of the 
# "intermediate"  and "far" contacts with respect to household contacts
infection_risk_contact_type_weights = {
        'very_far': 0, 
        'far': far_contact_weight, 
        'intermediate': intermediate_contact_weight,
        'close': 1}


# create the agent dictionaries based on the given parameter values and
# prevention measures
agent_types = compose_agents(measures)

# conduct all runs for an ensemble with a given set of parameters
ensemble_results = pd.DataFrame()
for run in range(1, N_runs + 1):

    f.value = c # update the progress bar
    c += 1

    # load the contact graph: since households and sibling contacts
    # are random, there are a number of randomly created instances of 
    # calibration schools from which we can chose. We use a different
    # calibration school instance for every run here
    G = nx.readwrite.gpickle.read_gpickle('../data/nursing_home/interactions_single_quarter.bz2')



    # initialize the model
    model = SEIRX_nursing_home(G, 
                               model_params['verbosity'], 
                               base_transmission_risk = model_params['base_risk'], 
                               testing = measures['testing'],
                               exposure_duration = model_params['exposure_duration'],
                               time_until_symptoms = model_params['time_until_symptoms'],
                               infection_duration = model_params['infection_duration'],
                               quarantine_duration = measures['quarantine_duration'],
                               subclinical_modifier = model_params['subclinical_modifier'], # literature                 
                               infection_risk_contact_type_weights = \
                               infection_risk_contact_type_weights , 
                                     K1_contact_types = measures['K1_contact_types'],
                               diagnostic_test_type = measures['diagnostic_test_type'],
                               preventive_screening_test_type = measures['preventive_screening_test_type'],
                               follow_up_testing_interval = \
                               measures['follow_up_testing_interval'],
                               liberating_testing = measures['liberating_testing'],
                               index_case = model_params['index_case'],
                               agent_types = agent_types, 
                               mask_filter_efficiency = model_params['mask_filter_efficiency'],
                               transmission_risk_ventilation_modifier = measures['ventilation_modification'],)


    # run the model until the outbreak is over
    for i in range(N_steps):

        # break if first outbreak is over
        if len([a for a in model.schedule.agents if \
            (a.exposed == True or a.infectious == True)]) == 0:
            break
        model.step()

        
    # get all data from the model
    data = model.datacollector.get_model_vars_dataframe()


       
    # delete data before testing
    # sometimes index cases are missed and we do not see them in empirical data
    if data.empty == False:
        if sum(data['N_diagnostic_tests'] > 0) > 0:
            data = data.loc[data[data['N_diagnostic_tests'] > 0].index[0]:]
            # wait  two days for results of PCR test
            data = data.iloc[3:]

        columns = ["run", "i", "I_employee", "I_resident", "E_employee", \
                   "E_resident", "R_employee", "R_resident"]

        data0 = np.array([np.repeat(run, len(data['I_employee']+1), axis=0),\
                          list(range(0, len(data['I_employee']))),\
                         data['I_employee'],data['I_resident'],\
                         data['E_employee'], data['E_resident'],\
                         data['R_employee'], data['R_resident']])
        df = pd.DataFrame(data=data0.transpose(), columns=columns)

        frames = [results, df]

        results = pd.concat(frames)

results.to_csv('Final'+(str(intermediate_contact_weight) + '-'+ str(far_contact_weight)+\
                ' Outbreak_sizes.csv'), index=False)

IntProgress(value=0, max=10000)